In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Together
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("TOGETHER_API_KEY")

f:\luat hinh su RAG Project\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embedding_model = HuggingFaceEmbeddings(
    model_name="dangvantuan/vietnamese-document-embedding",
    model_kwargs={'trust_remote_code': True,
                  'device': 'cuda'}
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_19852\924468509.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [3]:
vectorstore = Chroma(persist_directory='vectorstore/db_chroma',
                     embedding_function=embedding_model)

C:\Users\DELL\AppData\Local\Temp\ipykernel_19852\1385183956.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory='vectorstore/db_chroma',


In [4]:
len(vectorstore.get()['documents'])

917

In [5]:
vectorstore.get('67af9add-adc6-4117-a6f1-0828f0823df8')['documents']

[]

In [6]:
retriever = vectorstore.as_retriever(search_type = 'mmr',
                                     search_kwargs = {'k':3,
                                     'lambda_mult':0.7})

In [27]:
TEMPLATE = """Bạn là một trợ lý hữu ích, chuyên cung cấp thông tin dựa trên văn bản pháp luật.
Sử dụng CHỈ thông tin trong ngữ cảnh (context) được cung cấp dưới đây để trả lời câu hỏi của người dùng.
Tổng hợp và tóm tắt thông tin liên quan từ ngữ cảnh một cách súc tích và trực tiếp.
Nếu ngữ cảnh không chứa đủ thông tin để trả lời câu hỏi, hãy nói "Tôi không tìm thấy thông tin liên quan trong tài liệu được cung cấp."
KHÔNG sử dụng bất kỳ thông tin nào bên ngoài ngữ cảnh được cung cấp.
Bắt đầu câu trả lời ngay sau nhãn "Answer:".

Context: {context}
Question: {question}
Answer:"""

promt_template = PromptTemplate.from_template(TEMPLATE)

In [32]:
llm = Together(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    temperature=0.5,
    max_tokens=512,
)


C:\Users\DELL\AppData\Local\Temp\ipykernel_19852\1157748603.py:1: LangChainDeprecationWarning: The class `Together` was deprecated in LangChain 0.0.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-together package and should be used instead. To use it run `pip install -U :class:`~langchain-together` and import as `from :class:`~langchain_together import Together``.
  llm = Together(


In [33]:
chain = ({'context':retriever,
          'question':RunnablePassthrough()}
          | promt_template
          | llm
          | StrOutputParser()
)

In [34]:
question = "Tội xúi giục hoặc giúp người khác tự sát"

In [35]:
response = chain.invoke(question)

In [36]:
print(response)

 Người nào thực hiện một trong các hành vi sau đây, thì bị phạt cải tạo không giam giữ đến 03 năm hoặc phạt tù từ 06 tháng đến 03 năm: a) Kích động, dụ dỗ, thúc đẩy người khác tự tước đoạt tính mạng của họ; b) Tạo điều kiện vật chất hoặc tinh thần cho người khác tự tước đoạt tính mạng của họ. Phạm tội làm 02 người trở lên tự sát, thì bị phạt tù từ 02 năm đến 07 năm.
